In [21]:
%load_ext autoreload
%autoreload 2

import sys; sys.path.append("../../")

from copy import deepcopy

import numpy as np
import pandas as pd
import torch
from evobench.discrete import Trap
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.utilities.seed import seed_everything
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from hell import Surrogate, SurrogateData

seed_everything(42)

Global seed set to 42
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


42

In [22]:
benchmark = Trap(blocks=[3] * 3, verbose=1)

x_preprocessing = Pipeline([
    ("standard-scaler", StandardScaler())
])

y_preprocessing = Pipeline([
    ("min-max-scaler", MinMaxScaler())
])

data = SurrogateData(
    benchmark,
    x_preprocessing, y_preprocessing,
    n_samples=5e3, splits=(0.6, 0.2, 0.2),
    batch_size=5,
)

In [23]:
# ? sanity check

from xgboost import XGBRegressor
from sklearn.metrics import r2_score

xgb_model = XGBRegressor(n_estimators=100)
xgb_model.fit(data.x_train, data.y_train)
y_pred = xgb_model.predict(data.x_test)
r2_score(data.y_test, y_pred)


100%|██████████| 3000/3000 [00:00<00:00, 76922.07it/s]

Evaluating population of 3000 solutions


Initializing poptulation: 1000 samples

100%|██████████| 1000/1000 [00:00<00:00, 66666.20it/s]
Evaluating population of 1000 solutions




0.9999217322051477

In [24]:
surrogate = Surrogate(
    benchmark.genome_size,
    x_preprocessing, y_preprocessing,
    n_layers=0, learning_rate=1e-3, weight_decay=1e-6
)

early_stop_callback = EarlyStopping(
   monitor="val/r2",
   min_delta=0.000,
   patience=3,
   verbose=False,
   mode="max"
)

trainer = Trainer(
    max_epochs=100,
    gpus=1,
    progress_bar_refresh_rate=50,
    # callbacks=[early_stop_callback]
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [20]:
trainer.fit(surrogate, data.data_module)
trainer.test(surrogate)
surrogate.eval()

m=15]
Epoch 61:  75%|███████▌  | 600/800 [00:04<00:01, 143.75it/s, loss=0.0191, v_num=15]
Validating: 0it [00:00, ?it/s]
Epoch 62:  75%|███████▌  | 600/800 [00:04<00:01, 142.48it/s, loss=0.0206, v_num=15]
Validating: 0it [00:00, ?it/s]
Epoch 63:  75%|███████▌  | 600/800 [00:04<00:01, 145.81it/s, loss=0.0222, v_num=15]
Validating: 0it [00:00, ?it/s]
Epoch 64:  75%|███████▌  | 600/800 [00:04<00:01, 142.28it/s, loss=0.0213, v_num=15]
Validating: 0it [00:00, ?it/s]
Epoch 65:  75%|███████▌  | 600/800 [00:04<00:01, 147.86it/s, loss=0.0185, v_num=15]
Validating: 0it [00:00, ?it/s]
Epoch 66:  75%|███████▌  | 600/800 [00:04<00:01, 149.78it/s, loss=0.0172, v_num=15]
Validating: 0it [00:00, ?it/s]
Epoch 67:  75%|███████▌  | 600/800 [00:04<00:01, 149.70it/s, loss=0.0252, v_num=15]
Validating: 0it [00:00, ?it/s]
Epoch 68:  75%|███████▌  | 600/800 [00:04<00:01, 147.46it/s, loss=0.0191, v_num=15]
Validating: 0it [00:00, ?it/s]
Epoch 69:  75%|███████▌  | 600/800 [00:04<00:01, 144.27it/s, loss=0.0216, 

Surrogate(
  (linears): ModuleList(
    (0): Linear(in_features=9, out_features=4, bias=True)
  )
  (dropouts): ModuleList(
    (0): Dropout(p=0.2, inplace=False)
  )
  (linear_n): Linear(in_features=4, out_features=1, bias=True)
)